# Building AI Agents with LangGraph and LLMs

This workshop guides you through creating an AI agent using LangGraph, a powerful framework for building stateful, reasoning-focused AI systems with explicit control flows.

## What are AI Agents?

AI agents are autonomous systems that perceive their environment, make decisions, and take actions to accomplish specific goals. Unlike simple language models that just respond to prompts, agents can:

1. **Plan and Reason**: Break down complex tasks into logical steps
2. **Use Tools**: Access external capabilities like search engines, databases, or APIs
3. **Maintain State**: Remember context and previous actions across interactions
4. **Make Decisions**: Choose appropriate actions based on their reasoning

## What is LangGraph?

LangGraph is a framework for building stateful, reasoning-focused AI systems with explicit control flows. It provides:

1. **Structured Reasoning**: Create explicit multi-step reasoning processes
2. **State Management**: Maintain and transform context between steps
3. **Directed Workflows**: Design clear execution paths between components
4. **Conditional Logic**: Implement decision-making capabilities

In this workshop, we'll build a web search agent to demonstrate these concepts in action.

In [ ]:
import os
from typing import TypedDict, List, Annotated, Literal
from dotenv import load_dotenv
from pydantic import BaseModel, Field
from typing import List, Literal, Optional, Union
from langchain_community.document_loaders import WebBaseLoader
import json
# Load environment variables from .env file
load_dotenv()

# LangGraph imports
from langgraph.graph import StateGraph, END

# LangChain imports
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain_google_genai import ChatGoogleGenerativeAI  # Import Gemini
from langchain_openai import ChatOpenAI  # Alternative: OpenAI
from langchain_core.prompts import ChatPromptTemplate

# For web search
from langchain_community.utilities import GoogleSearchAPIWrapper  # Option 1
from langchain_community.utilities import SerpAPIWrapper  # Option 2

## Setting Up the Environment

We'll start by importing the necessary libraries for our agent. These include:

- **LangGraph**: For creating our agent's reasoning workflow
- **LangChain**: For working with LLMs and tools
- **Pydantic**: For creating strongly typed data models
- **Web Search APIs**: To give our agent the ability to search the internet

In [ ]:
# Check if API keys are set
print("Google API key is", "set" if os.environ.get("GOOGLE_API_KEY") else "not set")
print("OpenAI API key is", "set" if os.environ.get("OPENAI_API_KEY") else "not set")
print("Google CSE ID is", "set" if os.environ.get("GOOGLE_CSE_ID") else "not set")
print("SerpAPI key is", "set" if os.environ.get("SERPAPI_API_KEY") else "not set")

## Checking API Keys

Before proceeding, we need to verify that the necessary API keys are available in our environment:

- **Google API Key**: For Gemini LLM access or Google search capabilities
- **OpenAI API Key**: Alternative LLM provider
- **Google CSE ID**: For Google Custom Search Engine
- **SerpAPI Key**: Alternative search engine API

The agent will use these APIs to access knowledge beyond its training data.

In [ ]:
class SearchResult(BaseModel):
    id: int
    title: str
    snippet: str 
    link: str
    relevance_score: Optional[float] = None
    full_content: Optional[str] = None
    
class AgentState(BaseModel):
    messages: List[Union[HumanMessage, AIMessage, SystemMessage]] = Field(description="The chat history")
    search_results: List[SearchResult] = Field(default_factory=list, description="The results from the web search")
    optimized_query: Optional[str] = Field(default=None, description="The LLM-optimized search query")
    
    class Config:
        arbitrary_types_allowed = True

In [ ]:
def analyze_question(state: AgentState) -> AgentState:
    """Entry point: Analyze the user's question and answer with the knowledge you have."""
    try:
        # Get the latest user message
        last_user_message = next(
            (msg.content for msg in reversed(state.messages) if isinstance(msg, HumanMessage)),
            ""
        )

        # INSTRUCTOR_NOTE: This prompt can be improved by students during the workshop
        answer_prompt = ChatPromptTemplate.from_messages([
            ("system", """
            You are a knowledgeable sports analyst specializing in basketball.
            
            When asked about sports games or events:
            1. Always provide specific information you know about the game/team - including dates, scores, locations, etc.
            2. Cite notable performances and key moments from games you're familiar with
            3. If you're uncertain of the most recent result, provide information about the last game you're confident about
            4. Include the actual score and match details for any game you reference
            
            IMPORTANT: Even without real-time data, you should provide factual information about 
            past games and specifics you know about, not just generic responses.
            """),
            ("human", "Answer this query as specifically as possible: {query}")
        ])
        # Invoke the LLM 
        question_response = llm.invoke(
            answer_prompt.format_messages(
                query=last_user_message
            )
        )
        
        # Store the response in the state object
        state.messages.append(AIMessage(content=question_response.content))
        
        return state
        
    except Exception as e:
        # Just return the original state if there was an error
        return state

## Basic Agent Implementation

Here we implement our first agent function that analyzes user questions and provides responses.

### Understanding Node Functions in LangGraph

In LangGraph, **nodes** are functions that:
1. Take the current state as input
2. Perform some processing or reasoning
3. Return a modified state as output

Our `analyze_question` function implements this pattern by:
1. Extracting the user's message from the state
2. Creating a prompt for the LLM to answer the question
3. Invoking the LLM to generate a response
4. Adding the response to the message history in the state
5. Returning the updated state

This simple approach handles basic questions but has limitations - the LLM can only answer based on its training data, without access to current information.

In [ ]:
# INSTRUCTOR_COMPLETE_THIS_IN_WORKSHOP
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature = 0.7)

## Initializing the Language Model

We'll use Google's Gemini model as our reasoning engine. LLMs in agent systems serve multiple roles:

1. **Core Reasoning**: Analyzing problems and planning solutions
2. **Tool Use**: Deciding when and how to use available tools
3. **Information Processing**: Extracting and synthesizing knowledge
4. **Response Generation**: Creating human-readable outputs

The choice of LLM impacts the agent's capabilities, with factors like context window size, reasoning abilities, and tool use proficiency all playing important roles.

In [ ]:
def is_done(state: AgentState) -> Literal["end"]:
    """Simple function to signal the end of the workflow"""
    return "end"

## Workflow Control Functions

LangGraph uses specialized functions to control the flow of execution through the agent graph.

Here we define `is_done`, a simple function that always returns "end" to signal that processing is complete and the workflow should terminate. In more complex agents, this function could implement conditional logic to determine whether more processing is needed.

In [ ]:
# INSTRUCTOR_COMPLETE_THIS_IN_WORKSHOP
# Create the graph with our extended functions
workflow_v1 = StateGraph(AgentState)

# Add our nodes
workflow_v1.add_node("analyze", analyze_question)  # New entry point

# Connect process to end
workflow_v1.add_conditional_edges(
    "analyze",
    is_done,
    {
        "end": END  # Always end after processing
    }
)

# Set the entry point to our new analysis node
workflow_v1.set_entry_point("analyze")

# Compile the graph
agent = workflow_v1.compile()

## Creating the Graph Structure

Now we'll build our first graph structure using LangGraph's `StateGraph`. This creates the workflow that our agent will follow.

### Key Components of StateGraph:

1. **Nodes**: Functions that process the state (`analyze_question`)
2. **Edges**: Connections between nodes, defining the flow of execution
3. **Conditional Edges**: Paths that depend on the state or routing functions
4. **Entry Point**: The starting node when the graph is executed

When compiled, the graph becomes a runnable agent that can process user queries through the defined workflow.

In [ ]:
query = "Who won US presidential election in 2024?"
initial_state = AgentState(messages=[HumanMessage(content=query)])

# Run the agent
final_state = agent.invoke(initial_state)

In [ ]:
# Display the conversation
for message in final_state['messages']:
    if isinstance(message, HumanMessage):
        print(f"Human: {message.content}")
    elif isinstance(message, AIMessage):
        print(f"AI: {message.content}")
    else:
        print(message)

In [ ]:
# First, let's implement our search-enabled agent components

# Choose your search tool implementation - uncomment the one you want to use
# Option 1: Using Google Search API
search_engine = GoogleSearchAPIWrapper(k=3)  # Get top 3 results

# Task Start
def analyze_query(state: AgentState) -> AgentState:
    """Entry point: Analyze the user's question and optimize the search query"""
    try:
        # Get the latest user message
        last_user_message = next(
            (msg.content for msg in reversed(state.messages) if isinstance(msg, HumanMessage)),
            ""
        )
        
        # Create your own prompt for query optimization
        optimization_prompt = ChatPromptTemplate.from_messages([
            # TODO: Write a system prompt that helps optimize search queries
            ("system", """

       """),
            ("human", "Optimize this search query: {query}")
        ])
        
        # Invoke the LLM to optimize the query
        optimization_response = llm.invoke(
            optimization_prompt.format_messages(
                query=last_user_message
            )
        )
        
        # Store the optimized query directly in the state object
        state.optimized_query = optimization_response.content.strip()
        
        return state
        
    except Exception as e:
        # Just return the original state if there was an error
        return state
# Task END

def perform_search(state: AgentState) -> AgentState:
    """Second node: Perform the web search based on optimized query"""
    try:
        # Use the optimized query directly from state attribute, or fall back to original query
        if state.optimized_query:
            search_query = state.optimized_query
        else:
            # Fallback to original query if optimization failed
            search_query = next(
                (msg.content for msg in reversed(state.messages) if isinstance(msg, HumanMessage)),
                ""
            )
        
        # Perform the search
        try:
            # Try to get search results
            search_results = search_engine.results(search_query, num_results=3)
        except Exception as search_error:
            # Handle common API errors
            print(f"Search API error: {str(search_error)}")
            # Create a dummy result to allow the workshop to continue
            search_results = [{
                'title': 'Search temporarily unavailable',
                'snippet': f'Could not retrieve search results for: {search_query}. '
                          f'Please check your API keys and configuration.',
                'link': 'https://example.com'
            }]
        
        # Convert to our Pydantic model
        results_list = []
        for i, result in enumerate(search_results, 1):
            results_list.append(
                SearchResult(
                    id=i,
                    title=result.get('title', 'No title'),
                    snippet=result.get('snippet', result.get('description', 'No content')),
                    link=result.get('link', result.get('url', 'No link'))
                )
            )
        
        # Update state with search results
        state.search_results = results_list
        return state
        
    except Exception as e:
        print(f"Error during search: {str(e)}")
        # Return the original state with an empty search results list
        state.search_results = []
        return state


def process_results(state: AgentState) -> AgentState:
    """Fourth node: Process the query and search results to generate a response"""
    try:
        # Get current date
        from datetime import datetime
        current_date = datetime.now().strftime("%B %d, %Y")
        
        # Format search results for the LLM
        search_results_text = ""
        
        if not state.search_results:
            search_results_text = "No search results available. The search API may be unavailable."
        else:
            for result in state.search_results:
                search_results_text += f"Result {result.id}:\nTitle: {result.title}\nSnippet: {result.snippet}\nSource: {result.link}\n\n"
                
                if hasattr(result, 'full_content') and result.full_content:
                    search_results_text += f"\nExtracted Content:\n{result.full_content}\n\n"
                
        # Create a prompt template for synthesis
        synthesis_prompt = ChatPromptTemplate.from_messages([
            ("system", f"""You are a basketball sports analyst answering questions based on search results.
            Today's date is {current_date}.
            """),
            ("human", "User query: {original_query}\n\nSearch results:\n{search_results}"),
        ])
        
        # Get the original query
        original_query = next(
            (msg.content for msg in state.messages if isinstance(msg, HumanMessage)),
            "Unknown query"
        )
        
        # Generate a response
        response = llm.invoke(
            synthesis_prompt.format_messages(
                original_query=original_query,
                search_results=search_results_text
            )
        )
        
        # Update the state with the assistant's response
        state.messages.append(AIMessage(content=response.content))
        return state
        
    except Exception as e:
        # Add a basic response if processing fails
        state.messages.append(AIMessage(content="I apologize, but I encountered an issue processing your request. Please try again."))
        return state

## Enhanced Agent with Web Search Capabilities

To make our agent more powerful, let's add web search capabilities. This transforms it from a system limited by training data to one that can access current information.

### Multi-Step Reasoning Architecture

We'll implement a more sophisticated workflow with multiple specialized components:

1. **Query Analysis**: Optimize the user's question for search effectiveness
2. **Search Execution**: Retrieve relevant information from the web
3. **Result Processing**: Synthesize information into a coherent response

This demonstrates a key pattern in LangGraph: breaking complex tasks into focused subtasks with dedicated reasoning at each step. By specializing each component, we get better performance than trying to accomplish everything in a single prompt.

In [ ]:
# INSTRUCTOR_COMPLETE_THIS_IN_WORKSHOP
def route_to_search(state: AgentState) -> Literal["search"]:
    """Route to the search node after query analysis"""
    return "search"

## Routing Functions in LangGraph

Routing functions determine the flow of execution through the graph. They analyze the current state and return a string identifier that specifies which node should be executed next.

Here, `route_to_search` always returns "search" - creating a simple linear flow from analysis to search. In more complex agents, these functions could implement sophisticated decision logic based on the state.

In [ ]:
# STUDENT_IMPLEMENTATION_SECTION_BEGIN
# Create the graph with our extended functions
workflow_v2 = StateGraph(AgentState)

# Set the entry point to our new analysis node
workflow_v2.set_entry_point("analyze")

# Compile the graph
agent_v2 = workflow_v2.compile()
# STUDENT_IMPLEMENTATION_SECTION_END

## Designing the Enhanced Agent Graph

Let's create a more advanced agent graph with our multi-step workflow.

### LangGraph Workflow Design

When designing LangGraph workflows, consider these principles:

1. **Single Responsibility**: Each node should perform a specific, focused task
2. **State Transformation**: Nodes transform the state in predictable ways
3. **Explicit Decision Points**: Use conditional edges to represent decision logic
4. **Error Handling**: Account for failures and edge cases

Our enhanced agent follows this pattern with explicit nodes for query analysis, search, and result processing.

In [ ]:
query = "Who won US presidential election in 2024??"
initial_state = AgentState(messages=[HumanMessage(content=query)])

# Run the agent
final_state = agent_v2.invoke(initial_state)

## Testing Our Enhanced Agent

Let's test our search-enabled agent with a specific query. The execution will follow these steps:

1. Initial state with user query is created
2. Query analysis optimizes the search terms
3. Search retrieves relevant information
4. Processing synthesizes a comprehensive response

This demonstrates the multi-step reasoning pattern in action.

In [ ]:
# This cell shows the full state object - you can run it to explore the complete state
# final_state

In [ ]:
# Display the conversation
for message in final_state['messages']:
    if isinstance(message, HumanMessage):
        print(f"Human: {message.content}")
    elif isinstance(message, AIMessage):
        print(f"AI: {message.content}")
    else:
        print(message)

In [ ]:
# STUDENT_IMPLEMENTATION_SECTION_BEGIN
# Choose your search tool implementation - uncomment the one you want to use

# Option 1: Using Google Search API
search_engine = GoogleSearchAPIWrapper(k=3)  # Get top 3 results
def analyze_query(state: AgentState) -> AgentState:

    return state

def fetch_full_content(url: str, max_length: int = 8000) -> str:
    """Fetch the full content of a webpage and return a truncated version with better error handling"""
    try:
        # Add user-agent to avoid some basic blocking
        loader = WebBaseLoader(
            web_paths=[url],
            header_template={"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"}
        )
        
        # Try to load the content
        docs = loader.load()
        
        # Check if we got any content
        if docs and docs[0].page_content.strip():
            content = docs[0].page_content
            
            # Truncate to avoid token limits
            return content[:max_length] + ("..." if len(content) > max_length else "")
        else:
            # Try an alternative approach - extract from the snippet directly
            print(f"No content found using WebBaseLoader for {url}, using fallback method")
            
            # For sports sites, often the snippet contains the most relevant information
            # Return a message indicating we're using the snippet from search results
            return "[Full content unavailable. Using search result snippet instead]"
            
    except Exception as e:
        print(f"Error fetching content from {url}: {str(e)}")
        
        # More detailed error handling
        if "timeout" in str(e).lower():
            return "[Content unavailable: Connection timed out]"
        elif "403" in str(e) or "forbidden" in str(e).lower():
            return "[Content unavailable: Access forbidden - website may block scraping]"
        elif "404" in str(e) or "not found" in str(e).lower():
            return "[Content unavailable: Page not found]"
        else:
            return f"[Content unavailable: {str(e)}]"

def perform_search(state: AgentState) -> AgentState:
    """Second node: Perform the web search based on optimized query"""
    try:
        # Use the optimized query directly from state attribute, or fall back to original query
        if state.optimized_query:
            search_query = state.optimized_query
        else:
            # Fallback to original query if optimization failed
            search_query = next(
                (msg.content for msg in reversed(state.messages) if isinstance(msg, HumanMessage)),
                ""
            )
        
        # Perform the search
        try:
            # Try to get search results
            search_results = search_engine.results(search_query, num_results=5)
        except Exception as search_error:
            # Handle common API errors
            print(f"Search API error: {str(search_error)}")
            # Create a dummy result to allow the workshop to continue
            search_results = [{
                'title': 'Search temporarily unavailable',
                'snippet': f'Could not retrieve search results for: {search_query}. '
                          f'Please check your API keys and configuration.',
                'link': 'https://example.com'
            }]
        
        # Convert to our Pydantic model
        results_list = []
        for i, result in enumerate(search_results, 1):
            results_list.append(
                SearchResult(
                    id=i,
                    title=result.get('title', 'No title'),
                    snippet=result.get('snippet', result.get('description', 'No content')),
                    link=result.get('link', result.get('url', 'No link'))
                )
            )
        
        # Update state with search results
        state.search_results = results_list
        return state
        
    except Exception as e:
        print(f"Error during search: {str(e)}")
        # Return the original state with an empty search results list
        state.search_results = []
        return state

def enrich_content(state: AgentState) -> AgentState:
    """Third node: Use LLM to fetch and extract relevant information from top search results"""
    try:
        # Get the original query to provide context
        original_query = next(
            (msg.content for msg in state.messages if isinstance(msg, HumanMessage)),
            "Unknown query"
        )
        
        # Only enrich the first result to keep it simple for workshop purposes
        if state.search_results and len(state.search_results) > 0:
            for i in range(min(2, len(state.search_results))):
                # Use the first search result
                result = state.search_results[i]
                
                # First, fetch the raw content
                raw_content = fetch_full_content(result.link)
                
                # If content was successfully fetched, use LLM to extract and summarize
                if raw_content and not raw_content.startswith("[Content unavailable"):
                    # Create a prompt for the LLM to extract and summarize
                    extraction_prompt = ChatPromptTemplate.from_messages([
                        ("system", """You are an expert at extracting and summarizing relevant information from web content.
                        
                        Given a web page's content and a user query, your task is to:
                        1. Identify the most relevant sections to the query
                        2. Extract all the key facts, data points, and insights
                        3. Organize the information in a clear, coherent structure
                        4. Summarize the content while preserving accuracy
                        5. Include any relevant technical details, statistics, or examples
                        
                        Focus on extracting ONLY information that addresses the user's query.
                        Structure your summary with appropriate headings and bullet points if needed.
                        Limit your response to the most important information (around 500-800 words)."""),
                        ("human", "User query: {query}\n\nWeb page title: {title}\nURL: {url}\n\nContent:\n{content}")
                    ])
                    
                    # Invoke the LLM to extract and summarize
                    extraction_response = llm.invoke(
                        extraction_prompt.format_messages(
                            query=original_query,
                            title=result.title,
                            url=result.link,
                            content=raw_content
                        )
                    )
                    
                    # Store the LLM's extracted and summarized content
                    result.full_content = extraction_response.content
                else:
                    # If content couldn't be fetched, note this in the result
                    result.full_content = "[Content could not be accessed or processed]"
            
        return state
        
    except Exception as e:
        # Add a note about the failure
        if state.search_results and len(state.search_results) > 0:
            state.search_results[0].full_content = f"[Error during content enrichment: {str(e)}]"
        return state

def process_results(state: AgentState) -> AgentState:
    """Fourth node: Process the query and search results to generate a response"""
    try:

        return state
        
    except Exception as e:
        # Add a basic response if processing fails
        state.messages.append(AIMessage(content="I apologize, but I encountered an issue processing your request. Please try again."))
        return state
# STUDENT_IMPLEMENTATION_SECTION_END

## Advanced Agent with Content Enrichment

### LLMs as Multi-Purpose Components

In modern agent architecture, LLMs serve multiple roles beyond final output generation. Our agent demonstrates this with multiple LLM-powered processes:

1. **Query Analysis and Optimization**: Before searching, the LLM analyzes the user's question to create more effective search terms.

2. **Content Enrichment**: After retrieving search results, the LLM processes web content to extract the most relevant information.

3. **Response Synthesis**: Finally, the LLM combines all information into a coherent, comprehensive answer.

This multi-stage approach is far more effective than attempting to accomplish everything in a single LLM call. The `fetch_full_content` function enables a key capability: retrieving detailed information from web pages to augment the limited snippets provided by search engines.

In [ ]:
# INSTRUCTOR_COMPLETE_THIS_IN_WORKSHOP
def route_to_search(state: AgentState) -> Literal["search"]:
    """Route to the search node after query analysis"""
    return "search"

def should_enrich_content(state: AgentState) -> Literal["enrich", "process"]:
    """Decide whether to enrich content or proceed to processing"""
    # For simplicity in this workshop, we'll always enrich content
    return "enrich"

def is_done(state: AgentState) -> Literal["end"]:
    """Simple function to signal the end of the workflow"""
    return "end"

## Dynamic Flow Control in LangGraph

These functions control the flow of execution in our agent graph:

1. **route_to_search**: Directs the flow from query analysis to search
2. **should_enrich_content**: Determines whether to enrich content or proceed directly to processing
3. **is_done**: Signals the end of the workflow

Conditional routing enables dynamic decision-making in LangGraph. While our current functions are simple, they could implement complex logic in production systems, such as:

- Skipping search for questions the LLM can answer directly
- Enriching only the most promising search results
- Triggering additional research for insufficient information

These decision points are where the "agent" aspect truly emerges - the system adapts its behavior based on the specific context and needs of each query.

In [ ]:
# STUDENT_IMPLEMENTATION_SECTION_BEGIN
# Create the graph with our extended functions
workflow = StateGraph(AgentState)


# Compile the graph
agent = workflow.compile()
# STUDENT_IMPLEMENTATION_SECTION_END

## Building the Complete Agent Workflow

### LangGraph Architecture Patterns

Our complete agent implementation demonstrates key LangGraph architecture patterns:

1. **Graph-Based Orchestration**: Explicit workflow with clearly defined stages
2. **Typed State Management**: Structured state passed and transformed between nodes
3. **Conditional Branching**: Decision points that determine execution flow
4. **Component Specialization**: Each node focused on a specific task

The workflow follows this pattern:
```
[Analyze] → [Search] → [Enrich] → [Process] → END
```

This approach offers several advantages over monolithic agent designs:

- **Maintainability**: Components can be improved independently
- **Transparency**: Reasoning steps can be traced and debugged
- **Specialization**: Prompts optimized for specific subtasks
- **Robustness**: Failure in one component doesn't break the entire system

These patterns are essential for building production-grade agent systems.

In [ ]:
# Initialize with a query about current NBA playoffs
query = "Who won US presidential election in 2024?"
initial_state = AgentState(messages=[HumanMessage(content=query)])

# Run the agent
final_state = agent.invoke(initial_state)

# Display the conversation
for message in final_state['messages']:
    if isinstance(message, HumanMessage):
        print(f"Human: {message.content}")
    elif isinstance(message, AIMessage):
        print(f"AI: {message.content}")
    else:
        print(message)

In [ ]:
# This cell shows the final state object - you can run it to explore the complete state
# final_state

## Putting It All Together: Testing the Complete Agent

### The Complete Agent Experience

Let's test our complete agent with a specific query. The execution will demonstrate the full workflow:

1. **Query Analysis**: Optimizing the search terms
2. **Search Execution**: Retrieving information from the web
3. **Content Enrichment**: Processing detailed content from web pages
4. **Response Synthesis**: Creating a comprehensive answer

### Beyond This Workshop

This workshop demonstrates fundamental patterns in LangGraph agent development, but many advanced capabilities are possible:

1. **Memory and Long-term Context**: Storing and retrieving information across sessions
2. **Multi-agent Systems**: Coordinating multiple specialized agents
3. **Planning and Reflection**: Adding meta-cognitive abilities to agents
4. **Feedback and Learning**: Improving agent performance over time
5. **User Adaptation**: Customizing behavior based on user preferences

By understanding the core concepts presented here, you're well-prepared to explore these more advanced topics.

In [ ]:
# Display the final state
final_state

## Workshop Summary and Key Takeaways

### Core Concepts Covered

1. **AI Agent Architecture**
   - Components, state management, and decision making
   - Structuring complex reasoning workflows

2. **LangGraph Framework**
   - StateGraph, nodes, edges, and conditional routing
   - State transformation pattern

3. **Multi-stage LLM Reasoning**
   - Specialized prompting for subtasks
   - Processing and synthesis patterns

4. **Enhanced Agent Capabilities**
   - Web search integration
   - Content enrichment
   - Response synthesis

### Next Steps for Learning

1. **Experiment with different LLMs** to compare their reasoning capabilities
2. **Add more tools** to your agent (e.g., calculators, weather APIs)
3. **Implement persistent memory** to maintain context across sessions
4. **Create testing scenarios** to evaluate your agent's performance
5. **Build domain-specific agents** for particular use cases

By mastering these concepts, you're well on your way to developing sophisticated AI agents that can assist with complex, multi-step tasks while maintaining context and making intelligent decisions.

## Structured Output Parsing with Pydantic

One common challenge in agent development is ensuring consistent, well-structured outputs from LLM calls. Let's demonstrate how to use LangChain's PydanticOutputParser to enforce structured outputs in our agent nodes.

This approach provides several benefits:

1. **Type Safety**: Ensures outputs match your expected schema
2. **Error Handling**: Gracefully handles parsing failures
3. **Prompt Engineering**: Automatically generates format instructions
4. **Consistency**: Standardizes outputs across different LLM providers

Let's implement a version of our query analysis component with structured outputs:

In [ ]:
# STUDENT_IMPLEMENTATION_SECTION_BEGIN
# Import structured output parsing components
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
# Note: We're importing directly from pydantic rather than langchain_core.pydantic_v1
from typing import List, Optional
# STUDENT_IMPLEMENTATION_SECTION_END

In [ ]:
# STUDENT_IMPLEMENTATION_SECTION_BEGIN
from pydantic import BaseModel, Field, validator
from typing import List

class SearchQuery(BaseModel):
    """Schema for a structured search query output"""
    original_query: str = Field(description="The original user query")
    optimized_query: str = Field(description="The optimized search query")
    key_terms: List[str] = Field(description="Key terms extracted from the query")
    query_intent: str = Field(description="The interpreted intent of the user's query")
    information_need: str = Field(description="What specific information the user is looking for")
    
    # Using model_validator instead of validator for Pydantic v2
    @validator('key_terms')
    def validate_key_terms(cls, key_terms):
        """Validate that key terms are not empty and are well-formed."""
        if not key_terms or len(key_terms) < 1:
            raise ValueError("Must identify at least one key term")
        return key_terms
        
    # Add model_config for Pydantic v2
    model_config = {
        "json_schema_extra": {
            "examples": [
                {
                    "original_query": "What was the last Kauno Zalgiris basketball game?",
                    "optimized_query": "Kauno Zalgiris latest basketball game result",
                    "key_terms": ["Kauno Zalgiris", "basketball", "latest game"],
                    "query_intent": "finding information about recent sports event",
                    "information_need": "details about the most recent basketball game played by Kauno Zalgiris"
                }
            ]
        }
    }
# STUDENT_IMPLEMENTATION_SECTION_END

In [ ]:
# STUDENT_IMPLEMENTATION_SECTION_BEGIN
from pydantic import BaseModel, Field, validator
from typing import List, Optional

class StructuredResponse(BaseModel):
    """Schema for a structured agent response"""
    answer: str = Field(description="The direct answer to the user's question")
    sources: List[str] = Field(description="List of sources used to formulate the answer")
    confidence_score: float = Field(description="Confidence score between 0.0 and 1.0")
    related_topics: Optional[List[str]] = Field(None, description="Related topics the user might be interested in")
    
    @validator('confidence_score')
    def validate_confidence(cls, score):
        """Validate confidence score is between 0 and 1."""
        if score < 0 or score > 1:
            raise ValueError("Confidence score must be between 0.0 and 1.0")
        return score
        
    # Add model_config for Pydantic v2
    model_config = {
        "json_schema_extra": {
            "examples": [
                {
                    "answer": "Kauno Zalgiris played their last game against Barcelona on March 12, 2023, losing 88-73.",
                    "sources": ["https://example.com/sports/basketball/zalgiris-barcelona-2023"],
                    "confidence_score": 0.9,
                    "related_topics": ["Zalgiris EuroLeague standing", "Upcoming Zalgiris games"]
                }
            ]
        }
    }
# STUDENT_IMPLEMENTATION_SECTION_END

In [ ]:
# STUDENT_IMPLEMENTATION_SECTION_BEGIN
def analyze_query_structured(state: AgentState) -> AgentState:
    """Enhanced entry point: Analyze the user's question with structured output"""
    try:
        # Get the latest user message
        last_user_message = next(
            (msg.content for msg in reversed(state.messages) if isinstance(msg, HumanMessage)),
            ""
        )
        
        # Setup our Pydantic parser
        parser = PydanticOutputParser(pydantic_object=SearchQuery)
        
        # Format instructions tell the LLM how to structure its response
        format_instructions = parser.get_format_instructions()
        
        # Create a prompt for query optimization with structured output
        optimization_prompt = ChatPromptTemplate.from_messages([
            ("system", 
             """You are a research assistant synthesizing information from search results.
            
            Synthesize all information into a comprehensive, well-structured response that:
            - Directly answers the user's question
            - Provides specific details and examples
            - Cites sources appropriately
            - Acknowledges limitations in the information
            
            If search results are unavailable, acknowledge this limitation and provide 
            general information about the topic based on your knowledge.
                        
            {format_instructions}
            
            First analyze the query, then provide your response in the exact JSON format specified.
            Your response should be informative, balanced, and tailored to the user's specific query."""),
            ("human", "Analyze this search query: {query}")
        ])
        
        # Invoke the LLM to analyze the query with structured output
        optimization_response = llm.invoke(
            optimization_prompt.format_messages(
                query=last_user_message,
                format_instructions=format_instructions
            )
        )
        
        # Parse the response into our Pydantic model
        try:
            structured_query = parser.parse(optimization_response.content)
            
            # Store the optimized query in the state
            state.optimized_query = structured_query.optimized_query
            
            # You could store the full structured output in the state if needed
            # state.structured_query_analysis = structured_query
        except Exception as parse_error:
            # Fallback handling if parsing fails
            state.optimized_query = last_user_message
        
        return state
        
    except Exception as e:
        # Return the original state if there was an error
        return state
# STUDENT_IMPLEMENTATION_SECTION_END

In [ ]:
# STUDENT_IMPLEMENTATION_SECTION_BEGIN
def process_results_structured(state: AgentState) -> AgentState:
    """Process results with structured output parsing"""
    try:
        # Setup our Pydantic parser for the response
        parser = PydanticOutputParser(pydantic_object=StructuredResponse)
        format_instructions = parser.get_format_instructions()
        
        # Format search results for the LLM
        search_results_text = ""
        sources = []
        
        if not state.search_results:
            search_results_text = "No search results available. The search API may be unavailable."
        else:
            for result in state.search_results:
                search_results_text += f"Result {result.id}:\nTitle: {result.title}\nSnippet: {result.snippet}\nSource: {result.link}\n\n"
                sources.append(result.link)
                
                if hasattr(result, 'full_content') and result.full_content:
                    search_results_text += f"\nExtracted Content:\n{result.full_content}\n\n"
        
        # Create a prompt template for synthesis with structured output
        synthesis_prompt = ChatPromptTemplate.from_messages([
         ("system", """You are an expert at extracting and summarizing relevant information from web content.
                        
                        Given a web page's content and a user query, your task is to:
                        1. Identify the most relevant sections to the query
                        2. Extract all the key facts, data points, and insights
                        3. Organize the information in a clear, coherent structure
                        4. Summarize the content while preserving accuracy
                        5. Include any relevant technical details, statistics, or examples
                        
                        Focus on extracting ONLY information that addresses the user's query.
                        Structure your summary with appropriate headings and bullet points if needed.
                        Limit your response to the most important information (around 500-800 words).
                        Make sure to follow this format:
                        {format_instructions}
                        First analyze the search results, then provide your response in the exact JSON format specified.
          
          """),
            ("human", "User query: {original_query}\n\nSearch results:\n{search_results}"),
        ])
        
        # Get the original query
        original_query = next(
            (msg.content for msg in state.messages if isinstance(msg, HumanMessage)),
            "Unknown query"
        )
        
        # Generate a structured response
        response = llm.invoke(
            synthesis_prompt.format_messages(
                original_query=original_query,
                search_results=search_results_text,
                format_instructions=format_instructions
            )
        )
        
        try:
            # Parse the structured response
            structured_response = parser.parse(response.content)
            
            # Format the final response to the user with attribution
            final_response = f"{structured_response.answer}\n\n"
            
            if structured_response.sources and len(structured_response.sources) > 0:
                final_response += "Sources:\n"
                for source in structured_response.sources:
                    final_response += f"- {source}\n"
            
            if structured_response.related_topics and len(structured_response.related_topics) > 0:
                final_response += "\nRelated topics you might be interested in:\n"
                for topic in structured_response.related_topics:
                    final_response += f"- {topic}\n"
                    
            # Add confidence indicator
            confidence_level = ""
            if structured_response.confidence_score >= 0.8:
                confidence_level = "high confidence"
            elif structured_response.confidence_score >= 0.5:
                confidence_level = "moderate confidence"
            else:
                confidence_level = "low confidence"
                
            final_response += f"\n(Response provided with {confidence_level})"
            
            # Update the state with the assistant's response
            state.messages.append(AIMessage(content=final_response))
            
        except Exception as parse_error:
            # Fallback to raw response if parsing fails
            state.messages.append(AIMessage(content=f"I found some information that might help: {response.content}"))
        
        return state
        
    except Exception as e:
        # Add a basic response if processing fails
        state.messages.append(AIMessage(content="I apologize, but I encountered an issue processing your request. Please try again."))
        return state
# STUDENT_IMPLEMENTATION_SECTION_END

In [ ]:
# STUDENT_IMPLEMENTATION_SECTION_BEGIN
# Create a workflow with structured output parsing
structured_workflow = StateGraph(AgentState)

# Add our enhanced nodes
structured_workflow.add_node("analyze", analyze_query_structured)  # Structured query analysis
structured_workflow.add_node("search", perform_search)             # Reuse existing search
structured_workflow.add_node("enrich", enrich_content)             # Add the enrich node
structured_workflow.add_node("process", process_results_structured) # Structured response

# Connect analyze to search
structured_workflow.add_conditional_edges(
    "analyze",
    route_to_search,
    {
        "search": "search"  # Always go to search after analysis
    }
)

# Connect search to enrich (use the should_enrich_content function)
structured_workflow.add_conditional_edges(
    "search",
    should_enrich_content,
    {
        "enrich": "enrich",   # Enrich the top search result
        "process": "process"  # Skip enrichment (not used in our simple flow)
    }
)

# Connect enrich to process
structured_workflow.add_edge("enrich", "process")

# Connect process to end
structured_workflow.add_conditional_edges(
    "process",
    is_done,
    {
        "end": END  # Always end after processing
    }
)

# Set the entry point
structured_workflow.set_entry_point("analyze")

# Compile the enhanced agent
structured_agent = structured_workflow.compile()
# STUDENT_IMPLEMENTATION_SECTION_END

In [ ]:
# STUDENT_IMPLEMENTATION_SECTION_BEGIN
# Test the structured workflow with a query
query = "Who is in 2025 NBA playoffs?"
initial_state = AgentState(messages=[HumanMessage(content=query)])

# Run the enhanced agent with structured output parsing
final_state = structured_agent.invoke(initial_state)

# Display the conversation
for message in final_state['messages']:
    if isinstance(message, HumanMessage):
        print(f"Human: {message.content}")
    elif isinstance(message, AIMessage):
        print(f"AI: {message.content}")
    else:
        print(message)
# STUDENT_IMPLEMENTATION_SECTION_END

## Benefits of Structured Output Parsing

### Improved Agent Robustness

As demonstrated above, using structured output parsing with Pydantic provides several advantages for our agent:

1. **Clear Output Expectations**: The LLM knows exactly what format to produce
2. **Schema Validation**: Automatic validation prevents malformed outputs  
3. **Error Recovery**: Graceful fallbacks when parsing fails
4. **Extensibility**: Easy to add new fields as requirements evolve

### Real-World Applications

Structured outputs are particularly valuable in production systems where:

- Multiple services need to consume agent outputs
- Responses need to be validated before downstream usage
- You need consistent formatting across multiple LLM providers
- Outputs must be logged or stored in databases

The combination of LangGraph's structured state management and LangChain's output parsing capabilities creates a powerful foundation for building robust, maintainable agent systems.